In [ ]:
pip install datasketch
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import kagglehub

path = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")

print("Path to dataset files:", path)

c:\Users\daoto\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.06G/1.06G [00:31<00:00, 36.2MB/s]

Extracting files...


Path to dataset files: C:\Users\daoto\.cache\kagglehub\datasets\mohamedbakhet\amazon-books-reviews\versions\1


In [ ]:
import pandas as pd
from collections import defaultdict
import hashlib
import re
from datasketch import MinHash
from datasketch import MinHashLSH

In [ ]:
#import data
df_data_raw = pd.read_csv("books_data.csv")
df_rating_raw = pd.read_csv("Books_rating.csv")
  
df_data_raw = df_data_raw.drop(["image", "previewLink",'infoLink'], axis = 1)

In [11]:
df_data_raw[1:5]

,Title,description,authors,publisher,publishedDate,categories,ratingsCount
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,2005-01-01,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],NaN,2000,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],iUniverse,2005-02,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,2003-03-01,NaN,NaN


In [ ]:
df_rating_raw['review/text'][1]  

"I don't care much for Dr. Seuss but after reading Philip Nel's book I changed my mind--that's a good testimonial to the power of Rel's writing and thinking. Rel plays Dr. Seuss the ultimate compliment of treating him as a serious poet as well as one of the 20th century's most interesting visual artists, and after reading his book I decided that a trip to the Mandeville Collections of the library at University of California in San Diego was in order, so I could visit some of the incredible Seuss/Geisel holdings they have there.There's almost too much to take in, for, like William Butler Yeats, Seuss led a career that constantly shifted and metamoprhized itself to meet new historical and political cirsumstances, so he seems to have been both a leftist and a conservative at different junctures of his career, both in politics and in art. As Nel shows us, he was once a cartoonist for the fabled PM magazine and, like Andy Warhol, he served his time slaving in the ad business too. All was in

In [ ]:
#Preprocessing 
def clean_text(text):
    text = str(text).lower()               # Lowercase
    text = re.sub(r'[^\w\s]', '', text)    # Remove punctuation
    text = re.sub(r'\s+', ' ', text)       # Replace multiple spaces with one
    return text.strip()                    # Trim leading/trailing whitespace

df_rating_raw['cleaned_text'] = df_rating_raw['review/text'].apply(clean_text)


'i dont care much for dr seuss but after reading philip nels book i changed my mindthats a good testimonial to the power of rels writing and thinking rel plays dr seuss the ultimate compliment of treating him as a serious poet as well as one of the 20th centurys most interesting visual artists and after reading his book i decided that a trip to the mandeville collections of the library at university of california in san diego was in order so i could visit some of the incredible seussgeisel holdings they have theretheres almost too much to take in for like william butler yeats seuss led a career that constantly shifted and metamoprhized itself to meet new historical and political cirsumstances so he seems to have been both a leftist and a conservative at different junctures of his career both in politics and in art as nel shows us he was once a cartoonist for the fabled pm magazine and like andy warhol he served his time slaving in the ad business too all was in the service of amusing a

In [17]:
cleaned_df = df_rating_raw[['Title', 'cleaned_text']].copy()
cleaned_df

,Title,cleaned_text
0,Its Only Art If Its Well Hung!,this is only for julie strain fans its a colle...
1,Dr. Seuss: American Icon,i dont care much for dr seuss but after readin...
2,Dr. Seuss: American Icon,if people become the books they read and if th...
3,Dr. Seuss: American Icon,theodore seuss geisel 19041991 aka quotdr seus...
4,Dr. Seuss: American Icon,philip nel dr seuss american iconthis is basic...
...,...,...
2999995,The Idea of History,this is an extremely difficult book to digest ...
2999996,The Idea of History,this is pretty interesting collingwood seems l...
2999997,The Idea of History,this is a good book but very esoteric what is ...
2999998,The Idea of History,my daughter a freshman at indiana university e...


In [ ]:
# Choosing shingle size either 3 or 5 works well, test them out
#Comparing semantic similarity between documents.

#Cases where word order matters (e.g., plagiarism detection) -> word shingle instead of char shingle
def get_shingles(text, k):
    words = text.split()
    return [' '.join(words[i:i+k]) for i in range(len(words)-k+1)]

#sha1 is deterministic, so it is easier to reproduce the results -> stability
# Less prone to collisions than ad-hoc hashing methods.
def shingle_hash(shingle):
    return int(hashlib.sha1(shingle.encode()).hexdigest()[:8], 16)

In [ ]:
cleaned_df['shingles_5'] = cleaned_df['cleaned_text'].apply(lambda x: get_shingles(x, k = 5))
cleaned_df['hashed_shingles_5'] = cleaned_df['shingles_5'].apply(lambda x: [shingle_hash(sh) for sh in x])


print(cleaned_df[['Title', 'hashed_shingles_5' ]])

                                     Title  \
0           Its Only Art If Its Well Hung!   
1                 Dr. Seuss: American Icon   
2                 Dr. Seuss: American Icon   
3                 Dr. Seuss: American Icon   
4                 Dr. Seuss: American Icon   
...                                    ...   
133370  Hawkman: Allies & Enemies (Book 2)   
133371  Hawkman: Allies & Enemies (Book 2)   
133372        The Times Atlas of the World   
133373               Musical Movie Posters   
133374               Musical Movie Posters   

                                        hashed_shingles_5  \
0       [2497072899, 2613963973, 4028150693, 161550160...   
1       [2459356444, 3062230966, 3396066220, 257853039...   
2       [1563604061, 204419504, 1965917130, 54088906, ...   
3       [2791380534, 228220448, 190413931, 3498971767,...   
4       [995745743, 2425824490, 2729556045, 2873955957...   
...                                                   ...   
133370  [3471353009,

In [36]:
cleaned_df.to_csv('new_data.csv', index=False)

In [8]:
cleaned_df = pd.read_csv("new_data.csv")
cleaned_df = cleaned_df.reset_index()

In [ ]:
#sort shingle for min hash optimization step
cleaned_df = cleaned_df.sort_values('hashed_shingles_5')


In [26]:
len(cleaned_df)

133297

In [10]:
#not counted for the process since the amount of instances with less than 5 is negligable 78
#for 3 is 29
cleaned_df['less_5'] = cleaned_df['cleaned_text'].str.split().str.len() < 5

num_short = cleaned_df['less_5'].sum()
print(num_short)

78


In [ ]:
#remove reviews w less than 5 words
cleaned_df = cleaned_df[cleaned_df['cleaned_text'].str.split().str.len() >= 5]
cleaned_df

,index,Title,cleaned_text,shingles,hashed_shingles,shingles_5,hashed_shingles_5,shingles_3,hashed_shingles_3,less_5
85782,85782,"ERAGON: INHERITANCE, BOOK ONE.",im not going to go into detail about how erago...,"['im not going to go', 'not going to go into',...","[100000346, 216711606, 2502898856, 1888719204,...","['im not going to go', 'not going to go into',...","[100000346, 216711606, 2502898856, 1888719204,...","['im not going', 'not going to', 'going to go'...","[1683552948, 838612451, 1884425973, 3643246710...",False
112096,112096,Dreamcatcher Movie-Tie In,im not going to go into a detailed plot descri...,"['im not going to go', 'not going to go into',...","[100000346, 216711606, 3360644828, 3311500925,...","['im not going to go', 'not going to go into',...","[100000346, 216711606, 3360644828, 3311500925,...","['im not going', 'not going to', 'going to go'...","[1683552948, 838612451, 1884425973, 3643246710...",False
131499,131499,Dictionary of Nature Myths: Legends of the Ear...,a wonderful reference the book is well written...,"['a wonderful reference the book', 'wonderful ...","[100013018, 1496319068, 3229527655, 3282201912...","['a wonderful reference the book', 'wonderful ...","[100013018, 1496319068, 3229527655, 3282201912...","['a wonderful reference', 'wonderful reference...","[2231631001, 3355201767, 567139926, 1646706957...",False
94017,94017,From Abdullah to Hussein: Jordan in Transition...,for nearly six years from the assassination of...,"['for nearly six years from', 'nearly six year...","[1000179234, 2500261827, 2598613329, 417205320...","['for nearly six years from', 'nearly six year...","[1000179234, 2500261827, 2598613329, 417205320...","['for nearly six', 'nearly six years', 'six ye...","[3196468086, 3018119033, 1474528647, 200146242...",False
115667,115667,Cisco Internetworking and Troubleshooting,i cannot praise this book highly enough it is ...,"['i cannot praise this book', 'cannot praise t...","[1000186041, 173850130, 2771374928, 2548944534...","['i cannot praise this book', 'cannot praise t...","[1000186041, 173850130, 2771374928, 2548944534...","['i cannot praise', 'cannot praise this', 'pra...","[2885667614, 889295963, 3243341931, 3671066578...",False
...,...,...,...,...,...,...,...,...,...,...
29341,29341,Of Mice and Men (Penguin Audiobooks),this is a story about george and lennie george...,"['this is a story about', 'is a story about ge...","[999919860, 377732442, 1068894553, 1523056954,...","['this is a story about', 'is a story about ge...","[999919860, 377732442, 1068894553, 1523056954,...","['this is a', 'is a story', 'a story about', '...","[2855968966, 1161120632, 1018527723, 270696434...",False
128725,128725,Gone with the Wind (The Margaret Mitchell Anni...,this is a story about the life of a spoiled se...,"['this is a story about', 'is a story about th...","[999919860, 3885977566, 1489475564, 1801591445...","['this is a story about', 'is a story about th...","[999919860, 3885977566, 1489475564, 1801591445...","['this is a', 'is a story', 'a story about', '...","[2855968966, 1161120632, 1018527723, 373222847...",False
25881,25881,Candide and Other Writings,this is a story about the pains of truth and m...,"['this is a story about', 'is a story about th...","[999919860, 3885977566, 930077135, 2360560388,...","['this is a story about', 'is a story about th...","[999919860, 3885977566, 930077135, 2360560388,...","['this is a', 'is a story', 'a story about', '...","[2855968966, 1161120632, 1018527723, 373222847...",False
130362,130362,East of Eden,east of eden takes the biblical story of cain ...,"['east of eden takes the', 'of eden takes the ...","[999929506, 1816082015, 282888817, 747336951, ...","['east of eden takes the', 'of eden takes the ...","[999929506, 1816082015, 282888817, 747336951, ...","['east of eden', 'of eden takes', 'eden takes ...","[2429883074, 3674995489, 3225603845, 30156701,...",False


In [ ]:
# Minhash signature implementation
# a trade-off between accuracy and computational cost, according to the graph https://ekzhu.com/datasketch/minhash.html, 110 is a balance number for mh acc and mh performance
review_minhashes = {}

for _, row in cleaned_df.iterrows():
    mh = MinHash(num_perm=110)
    for shingle in row['hashed_shingles_5']:
        mh.update(str(shingle).encode('utf8'))
    review_minhashes[row['index']] = mh 

In [89]:
import pickle

with open('review_minhashes.pkl', 'wb') as f:
    pickle.dump(review_minhashes, f)

In [14]:
review_minhashes

{85782: <datasketch.minhash.MinHash at 0x28b8f0ad520>,
 112096: <datasketch.minhash.MinHash at 0x28b8f0aeff0>,
 131499: <datasketch.minhash.MinHash at 0x28abb5828a0>,
 94017: <datasketch.minhash.MinHash at 0x28abb5612e0>,
 115667: <datasketch.minhash.MinHash at 0x28b8f0b37a0>,
 96825: <datasketch.minhash.MinHash at 0x28abb51ce30>,
 36292: <datasketch.minhash.MinHash at 0x28b8f0ac950>,
 103918: <datasketch.minhash.MinHash at 0x28b8f0dc710>,
 15006: <datasketch.minhash.MinHash at 0x28abc827500>,
 53613: <datasketch.minhash.MinHash at 0x28b8f087d10>,
 81710: <datasketch.minhash.MinHash at 0x28b8f087560>,
 33783: <datasketch.minhash.MinHash at 0x28b8f0b3050>,
 11887: <datasketch.minhash.MinHash at 0x28b8f0b2e70>,
 89420: <datasketch.minhash.MinHash at 0x28c1cf71340>,
 46712: <datasketch.minhash.MinHash at 0x28c1cf712b0>,
 129171: <datasketch.minhash.MinHash at 0x28c1cf71220>,
 110193: <datasketch.minhash.MinHash at 0x28c1cf71070>,
 22691: <datasketch.minhash.MinHash at 0x28c1cf71280>,
 881

In [ ]:
#compute LSH n = 110, b = 22, r = 5 -> t about 0.53 dont care about false negative
# LSH Banding Implementation
from datasketch import MinHashLSH

lsh = MinHashLSH(threshold=0.53, num_perm=110)  # num_perm = b*r = 22*5 = 110

for review, mh in review_minhashes.items():
    lsh.insert(review, mh)

candidate_pairs = set()
for review in review_minhashes:
    matches = lsh.query(review_minhashes[review])
    for match in matches:
        if match != review:
            candidate_pairs.add(tuple(sorted((review, match))))



In [25]:
len(candidate_pairs)


499500

In [ ]:
index_pairs = [(9649, 34611),
 (2809, 42210),
 (102574, 131549),
 (121877, 125727),
 (13042, 121955),
 (19709, 92736),
 (6988, 87860),
 (32202, 109439),
 (2492, 49858),
 (41494, 84257),
 (76843, 77179),
 (51632, 62287),
 (17079, 30611),
 (6665, 94562),
 (38695, 94562),
 (48801, 68173),
 (6567, 91775),
 (82860, 90646)]
flat_indices = [idx for pair in index_pairs for idx in pair]
cleaned_df.loc[flat_indices]["cleaned_text"]



9649      i really enjoyed this book with the exception ...
34611     i really enjoyed this book and recommend it fo...
2809      consulting the wise is a compilation of simula...
42210     i really enjoyed this book i think that it is ...
102574    gerrold and niven are both excellent sf writer...
131549    as in his seminars don denoncourt attempts to ...
121877    an engaging and emotional account of a young w...
125727    i have an avid young reader in my family and i...
13042     no disrespect october 29 2003the novel no disr...
121955    the story of a bugs lifeabout a clutz named fl...
19709     to give you some what of a background it took ...
92736     ive been a fan of this series for many years i...
6988      i really enjoyed this book kerouac drew me int...
87860     awesome read very descriptive and interesting ...
32202     i think up from slavery is a good book its for...
109439    soldier x is one of the best books ive ever re...
2492      es un libro de puntos muy astu